In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_csv('./SouthParkData-master/All-seasons.csv')
raw_data.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


In [3]:
raw_data['Line'][4]

"Chef said he's been bored, so he joining a group called the Super Adventure Club. \n"

## We're going to attempt to implement the Viterbi algorithm
From https://en.wikipedia.org/wiki/Viterbi_algorithm

### Here's what we need:

#### parameters:

K => number of hidden states

T => length of sequence of observations

N => number of possible observations 

S => the "state space", i.e. all possible words (s1, s2, ... , sK)

Priors => an array of prior probabilities for each state (ie. how likely is a word to occur w/o context?)

Transition Matrix A => K x K matrix where A[i, j] stores probability of transitioning from si to sj

Emission Matrix   B => K x N matrix where B[i, j] stores probability of observing oj from state si
    
#### output:

X - a sequence of states (x1, x2, ..., xT)

Let's first see how large of a vocabulary we're working with

I'm going to include all individual words, plus some basic punctuation like ',' '.' '!' and '?'

In [109]:
# Takes a string, tokenizes it 
# Returns a list of the tokens
def tokenize_str(string):
    
    string = string.lower()
    
    # remove punctuation except for newline
    string = string.replace(',', '')
    string = string.replace('.', '')
    string = string.replace('!', '')
    string = string.replace('?', '')
    string = string.replace('-', '')
    string = string.replace('(', '')
    string = string.replace(')', '')
    string = string.replace(';', '')
    
    # split string and add newline at end
    string = string.split()
    string += ['\n']
    
    return string

print(tokenize_str(raw_data['Line'][4]))

['chef', 'said', "he's", 'been', 'bored', 'so', 'he', 'joining', 'a', 'group', 'called', 'the', 'super', 'adventure', 'club', '\n']


In [110]:
# first I think I want one array of the entire corpus, this will be useful for all 3 of these remaining variables

def get_corpus(dialogue):
    # dialogue <=> a column of strings in a DataFrame

    return [word for sentence in dialogue for word in tokenize_str(sentence)]

corpus = get_corpus(raw_data['Line'][:500])

We would have a vocab of 32834 words, but I'm going to just use the first 500 lines of dialogue for this first go. We'll see if we can feasibly increase it later.

Since we're trying to generate text from this corpus, our # of observations N will be the same as K (vocab_size)

Now we have our states, and also our # of states K, and our # of observations N. In our application, K == N


In [6]:
states = list(set(corpus))
vocab_size = len(states)
corpus_size = len(corpus)
print("corpus size: {cs} \nvocab size: {vs}".format(cs=corpus_size, vs=vocab_size))

corpus size: 5836 
vocab size: 1164


1181 words will be more manageable

We still need the transition matrix A, our emission matrix B, and our array of priors for each observation

Let's do our array of priors, which I believe is simply the frequency of the words in the corpus

In [7]:
import collections # <= for counting the frequency of each word in the corpus (O(n) implementation)

def get_priors(corpus):
    counter = collections.Counter(corpus)
    words = list(counter.keys())
    probabilities = [c/len(corpus) for c in list(counter.values())]
    return dict(zip(words, probabilities))

priors = get_priors(corpus)

# now we have a priors array where the prior probability of word states[i] = priors[i]
print(priors['\n'])

0.08567511994516792


In [8]:
# I want the counts given a word for the next step so I'll define that here
def get_counts(corpus):
    counter = collections.Counter(corpus)
    return dict(zip(list(counter.keys()), list(counter.values())))

In [9]:
# I'm currently confused about what the transition matrix and emission matrix are, and if they're even different. 
# Since we only have the corpus to work with, (I assume this is where the transition matrix comes from),
# as long as I decide to include all words of the corpus in the observation space, the emission matrix
# should be the same as the transition matrix...? I could be wrong on this, but I'm going to roll with it

# So I'm going to essentially compute the probability of transitioning from one word to another w/o further context

def get_transition_matrix (corpus, states):
    K = len(states)
    N = len(corpus)
    
    # initialize with zeros
    transition_matrix = np.zeros((K, K))
    
    state_index = {word:i for i, word in enumerate(states)}

    for word_index in range(1, N):
        to = corpus[word_index]
        frm = corpus[word_index-1]
        
        transition_matrix[state_index[frm]][state_index[to]] += 1
    
    counts = get_counts(corpus)
    for frm in range(K):
        for to in range(K):
            transition_matrix[frm][to] /= counts[states[frm]]
            
    return transition_matrix
        
transition_matrix = get_transition_matrix(corpus, states)

In [10]:
i = 17
print(max(transition_matrix[i]))
print(states[i])
print(get_counts(corpus)[states[i]])

0.25
butters
8


# I messed up
For some reason I thought I'd be able to generate text using the Viterbi algorithm. It's actually the Forward-Backward algorithm I want to implement - we might try to use Viterbi later 

Luckily, the work done so far is still useful

So, 
# Implementing the Forward-Backward algorithm
From https://en.wikipedia.org/wiki/Forward%E2%80%93backward_algorithm



In [11]:
def forward_backward(observations, states, priors, transition_matrix, emission_matrix, end_state):
    '''
    Forward-Backward Algorithm
    
    observations - sequence of observations (list of strings) : this is the corpus
    states - set of hidden states (list of unique strings)
    priors - dictionary of prior probabilities of each state (string -> prior prob)
    transition_matrix - tm[i][j] gives prob of transitioning from states[i] to states[j]
    emission_matrix - em[i][j] gives prob of emitting observation j given hidden state states[i]
    '''
    
    
    K = len(observations)
    S = len(states)
    
    # forward algorithm
    forward = []
    
    # make a pass thru the observations to compute joint prob P(states[k], observations[1:k])
    for i, observation in enumerate(observations):
        f_current = {}
        for s in range(S):
            if i == 0:
                prev_sum = priors[states[s]]
            else:
                prev_sum = sum(f_prev[states[k]] * transition_matrix[k][s] for k in range(S))
            f_current[states[s]] = emission_matrix[s][states.index(observations[i])] * prev_sum

        forward.append(f_current)
        f_prev = f_current
        
    p_forward = sum(f_current[states[k]] * transition_matrix[k][states.index(end_state)] for k in range(S))
    
    # backward algorithm
    backward = []

    for i, observation in enumerate(reversed(observations[1:] + [None,])):
        b_current = {}
        for s in range(S):
            if i == 0:
                b_current[states[s]] = transition_matrix[s][states.index(end_state)]
            else:
                b_current[states[s]] = sum(transition_matrix[s][l] * emission_matrix[l][states.index(observations[i])] * b_prev[states[l]] for l in range(S))
        
        backward.insert(0, b_current)
        b_prev = b_current
    
    p_backward = sum(priors[states[l]] * emission_matrix[l][states.index(observations[0])] * b_current[states[l]] for l in range(S))
    
    posterior = []
    for i in range(len(observations)):
        posterior.append({state: forward[i][state] * backward[i][state] / p_forward for state in states})
        
    # assert p_forward == p_backward
    return forward, backward, posterior

small_corpus = get_corpus(raw_data['Line'][:50])
small_states = list(set(small_corpus))
small_priors = get_priors(small_corpus)
small_transition_matrix = get_transition_matrix(small_corpus, small_states)
print(len(small_corpus), len(small_states))


fwd, bkwd, post = forward_backward(small_corpus, small_states, small_priors, small_transition_matrix, small_transition_matrix.transpose(1,0), '\n')
print(max(post[1].values()))

428 172
nan


<ipython-input-11-a228e949654a>:52: RuntimeWarning: invalid value encountered in double_scalars
  posterior.append({state: forward[i][state] * backward[i][state] / p_forward for state in states})


### I'm having trouble keeping track of all the states vs. indices of the states so I'm going to make a HMM class

#### hopefully this improves understanding & clean up the code a little

In [12]:
class HMM():
    def __init__(self, corpus):
        self.Corpus = corpus
        self.States = self.get_states()
        self.Observations = self.States
        self.TransitionMatrix = self.dictify(self.get_transition_matrix())
        self.EmissionMatrix = self.dictify(self.get_transition_matrix())
        self.Priors = self.get_priors()
        
        
    def dictify(self, matrix2d):
        return {frm: {to: matrix2d[i][j] for j, to in enumerate(self.States)} for i, frm in enumerate(self.States)}
    
    
    # self.Corpus is assumed to be an ordered list of all words in the corpus 
    def get_states(self):
        return list(set(self.Corpus))
    
    
    def get_counts(self):
        counter = collections.Counter(self.Corpus)
        return dict(zip(list(counter.keys()), list(counter.values())))
        
        
    def get_transition_matrix(self):
        K = len(self.States)
        N = len(self.Corpus)

        # initialize with zeros
        transition_matrix = np.zeros((K, K))
        
        # iterate over observations and increase word transition counts
        for word_index in range(1, N):
            to = self.States.index(self.Corpus[word_index])
            frm = self.States.index(self.Corpus[word_index-1])

            transition_matrix[frm][to] += 1

        # divide by word count
        counts = self.get_counts()
        for frm in range(K):
            for to in range(K):
                transition_matrix[frm][to] /= counts[self.States[frm]]

        return self.laplace_smooth(transition_matrix, K)
    
    
    def get_priors(self):
        counter = collections.Counter(corpus)
        words = list(counter.keys())
        probabilities = [c/len(corpus) for c in list(counter.values())]
        return dict(zip(words, probabilities))
    
    # trying laplace smoothing after my forward-backward algorithm kept
    # running into division by zero
    def laplace_smooth(self, matrix, total_count):
        matrix[matrix != 0] = matrix[matrix != 0] + (1 / total_count)
        matrix[matrix == 0] = 1 / total_count
        return matrix

In [13]:
small_corpus = get_corpus(raw_data['Line'][:30])
Model = HMM(small_corpus)

In [14]:
print(Model.States)
print(Model.TransitionMatrix)


['choice', 'what', 'club', 'a', 'been', 'life', 'to', 'two', 'back', 'group', "don't", 'our', "i'm", 'the', 'super', 'go', 'hello', 'have', 'fatass', 'tell', "what's", 'on', 'chef', 'well', "it's", '\n', 'get', 'world', 'dude', 'boys', 'is', 'questions', 'goodbye', 'card', 'must', 'do', 'forever', 'how', 'long', 'why', 'here', 'but', 'meaning', 'answer', 'fuhfffriend', 'right', 'time', 'your', 'around', 'so', "that's", 'you', 'it', "he's", 'adventure', 'hope', 'was', 'will', 'believe', 'draw', 'and', 'reverse', 'gonna', 'for', 'with', 'yeah', 'kind', 'bored', 'byebye', 'guys', 'him', 'wow', 'sorry', 'true', 'miss', 'jew', 'making', 'tells', 'of', 'great', 'good', "you're", 'there', 'we', 'said', 'he', 'all', 'know', 'away', 'i', 'iand', "can't", 'heart', 'children', 'going', 'called', 'adventuring', 'joining', "i'll", 'think', 'are']
{'choice': {'choice': 0.009900990099009901, 'what': 0.009900990099009901, 'club': 0.009900990099009901, 'a': 0.009900990099009901, 'been': 0.0099009900990

In [15]:
def forward_backward(HMM, observations, end_state):
    
    # forward pass
    forward = []
    f_prev = {}
    
    # make a pass thru the observations to compute joint prob P(states[k], observations[1:k])
    for i, observation in enumerate(HMM.Observations):
        f_current = {}
        for state in HMM.States:
            if i == 0:
                prev_sum = HMM.Priors[state]
            else:
                prev_sum = sum(f_prev[k] * HMM.TransitionMatrix[k][state] for k in HMM.States)
            
            f_current[state] = HMM.EmissionMatrix[state][observation] * prev_sum

        forward.append(f_current)
        f_prev = f_current
        
    p_forward = sum(f_current[k] * HMM.TransitionMatrix[k][end_state] for k in HMM.States)
    print(p_forward)
    
        
    # backward algorithm
    backward = []

    for i, observation in enumerate(reversed(HMM.Observations[1:] + [None,])):
        b_current = {}
        for state in HMM.States:
            if i == 0:
                b_current[state] = HMM.TransitionMatrix[state][end_state]
            else:
                b_current[state] = sum(HMM.TransitionMatrix[state][l] * HMM.EmissionMatrix[l][observation] * b_prev[l] for l in HMM.States)
        
        backward.insert(0, b_current)
        b_prev = b_current
    
    p_backward = sum(HMM.Priors[l] * HMM.EmissionMatrix[l][HMM.Observations[0]] * b_current[l] for l in HMM.States)
    
    posterior = []
    for i in range(len(HMM.Observations)):
        posterior.append({state: forward[i][state] * backward[i][state] / p_forward for state in HMM.States})
        
    return posterior

In [17]:
medium_corpus = get_corpus(raw_data['Line'][:50])
MedModel = HMM(medium_corpus)
post = forward_backward(MedModel, medium_corpus, '\n')

1.474916268381536e-297


In [18]:
import operator

def generate_text(posteriors):
    wordPath = []
    for item in posteriors:
        highest_value = max(item.items(), key=operator.itemgetter(1))[0]
        wordPath.append(highest_value)

    for word in wordPath:
        print(word, end=' ')

generate_text(post)

i guess for also just what super adventure joining very chef be he's of belong to draw they interesting moved back i really well 
 was 
 what's the school the gonna 
 so old card to to must you little chef 
 you're it go on 
 we'll in hey well it i'll the that with 
 like you sorry buy world of 
 with you not still two you must do sound dude how long 
 life live 
 little uh see ya our that's great all adventuring bored so have get said super adventure friends i chef me can your we i'm can't 
 i chef 
 i'm was you care for park stay 
 what been south later can you tell get everybody 
 happy that i'm here it's gonna buy another house you're heart tells meaning a for you believe hello why are you chef you're so will don't going iand and i 
 your 
 guys there is group think oh he feeling very you back 

#### Alright... interesting... Let's try doubling the corpus size

In [92]:
corpus_2 = get_corpus(raw_data['Line'][:100])
Model2 = HMM(corpus_2)
post = forward_backward(Model2, corpus_2, '\n')

generate_text(post)

0.0
choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice choice ch

<ipython-input-15-c74f5ddbab7a>:43: RuntimeWarning: invalid value encountered in double_scalars
  posterior.append({state: forward[i][state] * backward[i][state] / p_forward for state in HMM.States})


# :(
I'm getting this consistent problem where the forward pass reduces p_forward to zero 

I'm currently having this crazy confusion about the distinction between the transition matrix and the emission matrix. Aren't they the same in this application?

Let's try Viterbi for text prediction now

In [41]:
#
# ! This section is a failed attempt at Viterbi ! 
# 

def viterbi(HMM, observations):
    O = len(observations)
    K = len(HMM.States)
    
    dynamic = np.zeros((K, O))
    
    # get each state's probability for observation zero
    for state in HMM.States:
        dynamic[HMM.States.index(state)][0] = HMM.Priors[state] * HMM.EmissionMatrix[state][observations[0]]
    
    for o in range(1, O):
        for state in HMM.States:
            state_index = HMM.States.index(state)
            k_max = np.argmax(dynamic[state_index][o-1] 
                         * HMM.TransitionMatrix[k][state] 
                         * HMM.EmissionMatrix[state][observations[o]] for k in states)
            for k in states:
                print(dynamic[state_index][o-1] * HMM.TransitionMatrix[k][state] * HMM.EmissionMatrix[state][observations[o]])
            print(k_max)
            k_max_state = HMM.States[k_max]
            print(k_max_state)
            dynamic[state_index][o] = (dynamic[k_max][o-1] 
                                                   * HMM.TransitionMatrix[k_max_state][state] 
                                                   * HMM.EmissionMatrix[state][observations[o]])
    
    best_path = []
    for o in range(-1, -O, -1):
        print(o)
        

print(MedModel.States)
viterbi(MedModel, np.array(['everybody', 'will', 'still']))

['choice', "we'll", 'deeply', 'live', 'what', 'school', 'they', 'club', 'a', 'happy', 'would', 'feeling', 'been', 'life', 'to', 'little', 'two', 'sound', 'people', 'back', 'group', "don't", 'interesting', 'come', 'later', 'our', "i'm", 'the', 'bet', 'tomorrow', 'super', 'go', 'hello', 'have', 'friends', 'fatass', 'stay', 'tell', 'be', 'decided', 'uh', 'seem', "what's", 'not', 'seems', 'on', 'chef', 'well', 'see', 'south', 'everybody', "it's", '\n', 'get', 'world', 'mean', 'new', 'dude', 'really', 'had', 'boys', 'another', 'is', 'questions', 'goodbye', 'me', 'can', 'in', 'belong', 'card', 'must', 'do', 'forever', 'like', 'how', 'long', 'nnono', 'thank', 'why', 'here', 'but', 'meaning', 'trippy', 'ya', 'answer', 'fuhfffriend', 'right', 'time', 'your', 'around', 'so', "that's", 'you', 'it', "he's", 'adventure', 'oh', 'whom', 'hope', 'was', 'until', 'still', 'old', 'will', 'sure', 'believe', 'draw', 'guess', 'and', 'reverse', 'gonna', 'just', 'buy', 'for', 'now', 'again', 'with', 'yeah', '

KeyError: 'alain'

In [91]:
# viterbi will find the most likely state sequence to have produced the given observations

# so input is some string of words (must come from the corpus)
# and output is the most likely hidden states which could've produced those words

def viterbi(HMM, observations):

    prob_table = np.zeros((len(observations), len(HMM.States)))
    prev_table = np.empty((len(observations), len(HMM.States)), dtype=object)
    
    # initial probabilities are the prior probability of the hidden state times the probability
    # of emitting the first observation we saw
    
    # we can use this base to find the most likely states given more observations 
    for s, state in enumerate(HMM.States):
        prob_table[0][s] = HMM.Priors[state] * HMM.EmissionMatrix[state][observations[0]]
        prev_table[0][s] = None
    
    # loop over the observations
    for o in range(1, len(observations)):
        
        # here we're using the previous most likely hidden state/observation probability to 
        # calculate the most likely hidden state for the next observation we see
        for s, state in enumerate(HMM.States):
            max_transition_prob = prob_table[o-1][0] * HMM.TransitionMatrix[HMM.States[0]][state]
            prev_state_selected = HMM.States[0]
            for prev_state in HMM.States[1:]:
                transition_prob = prob_table[o-1][HMM.States.index(prev_state)] * HMM.TransitionMatrix[prev_state][state]
                if transition_prob > max_transition_prob:
                    max_transition_prob = transition_prob
                    prev_state_selected = prev_state
            
            max_prob = max_transition_prob * HMM.EmissionMatrix[state][observations[o]]
            prob_table[o][s] = max_prob
            prev_table[o][s] = prev_state_selected

    
    # find most probable last word
    last_state_index = np.argmax(prob_table[-1])
    last_state = HMM.States[last_state_index]
    predictions = [last_state]
    
    # backtrack from most probable last word
    prev = last_state
    prev_index = last_state_index
    for back in range(len(prev_table) - 2, -1, -1):
        predictions.insert(0, prev_table[back+1][prev_index])
        prev_index = HMM.States.index(prev_table[back+1][prev_index])
    
    return predictions

predictions = viterbi(MedModel, ['everybody', 'will', 'still'])
print(predictions)

['hey', 'everybody', 'can']


### If I'm being honest, I have no idea if this is reasonable. As far as I can tell the algorithm is correct, but the predicted hidden states could be horrible. Probably, since the corpus is so small. 

Let's try a larger corpus for just Viterbi

In [95]:
def generate_viterbi_predictions(HMM, input_str):
    inp = tokenize_str(input_str)
    predictions = viterbi(HMM, inp)
    for word in predictions:
        print(word, ' ', end='')
        
generate_viterbi_predictions(Model2, "i'll hope it was interesting")


  i'll  get  arrested  all  right  

In [113]:
corpus3 = get_corpus(raw_data['Line'][:200])
Model3 = HMM(corpus3)
print("Corpus size: ", len(Model3.States))
print(Model3.States)

Corpus size:  514
['then', 'deeply', 'live', 'wait', 'even', 'difficult', 'whatever', 'years', 'chubby', 'feeling', 'may', 'waitresses', 'life', 'secret', 'to', 'maybe', 'vortex', 'offers', "we've", 'later', 'dance', 'bet', 'manual', "there's", 'arrested', 'friends', 'perhaps', 'fatass', 'months', 'tell', 'seem', "what's", 'course', 'seems', 'sir', 'better', 'pretty', 'nuts', 'get', 'hit', 'bitch', 'dr', 'aw', 'new', 'need', 'few', 'never', 'simple', 'had', 'another', 'goodbye', 'let', 'can', 'minute', 'forever', 'got', 'whrrrrrr', 'like', 'how', 'us', 'damage', 'trippy', 'ya', 'time', 'jarvis', 'gotta', 'coming', 'so', "that's", 'you', 'it', 'adventure', 'was', 'work', 'still', 'old', 'happened', 'this', 'and', 'kyle', 'ahh', 'together', 'having', 'before', 'p', 'with', 'weird', 'mysterious', 'molesting', 'bored', 'molest', 'out', 'byebye', 'also', 'sweet', 'moved', 'call', 'hunting', 'undo', 'indeed', 'white', 'if', 'sorry', 'kilimanjaro', 'true', 'rim', 'today', 'try', 'k2', 'saying

In [119]:
strings = [
    "whom is going adventuring",
    "why are you tiring",
    "answer me friends",
    "how long before we go",
    "thank you sir that's great",
    "you're here but not there",
]
def predict_hidden_states(model, strings):
    for s in strings:
        print("Input string: ", s)
        print("Generated string: ", end='')
        generate_viterbi_predictions(model, s)
        print("\n")
predict_hidden_states(Model3, strings)

Input string:  whom is going adventuring
Generated string: the  hell  is  going  away  

Input string:  why are you tiring
Generated string: life  why  are  pretty  tiring  

Input string:  answer me friends
Generated string: 
  excuse  me  too  

Input string:  how long before we go
Generated string: kenny  how  to  them  we  go  

Input string:  thank you sir that's great
Generated string: 
  tell  me  
  later  byebye  

Input string:  you're here but not there
Generated string: that  you  
  he's  pretty  tiring  



In [120]:
corpus4 = get_corpus(raw_data['Line'][:500])
Model4 = HMM(corpus4)
print("Corpus size: ", len(Model4.States))

Corpus size:  1153


In [121]:
import time

start = time.time()
predict_hidden_states(Model4, strings)
print("time to run: ", time.time() - start)

Input string:  whom is going adventuring
Generated string: the  point  is  pretty  tiring  

Input string:  why are you tiring
Generated string: life  why  do  pretty  tiring  

Input string:  answer me friends
Generated string: sex  with  new  neighbors  

Input string:  how long before we go
Generated string: desert  how  do  though  we  go  

Input string:  thank you sir that's great
Generated string: 
  tell  me  
  later  byebye  

Input string:  you're here but not there
Generated string: that  you  know  kyle's  movin'  away  

time to run:  139.66380834579468


### Again, if I'm being honest, I can't tell if this is correct. And the time it takes to predict seems to be growing quadratically (at least). 

# Summary
The implementation of the forward-backward algorithm was a half-failure. I couldn't scale it, but I did manage to generate a chunk of text from what I could implement. I still don't know what went wrong here. 

The Viterbi implementation was less of a failure. At least, as far as I can tell. I tweaked that algorithm many times before it seemed to work, and even then, the results are dubious and slow.

I wish I had encoded the words instead of storing them literally as dictionary with the words themselves as keys. I think it would've been faster if some sort of encoding could've been made.

This NLP implementation of a Hidden Markov Model was hard to wrap my mind around - the transition matrix seemed (and still seems) to be the EXACT same as the emission matrix. When our corpus is all we have to train on, the words have to act as both our hidden states and our observations. I don't see how else it could be. I look forward to keeping this in mind for other non-linguistic applications and seeing how it might perform. 